In [1]:
import mbuild as mb
from mbuild.recipes import Polymer

In [2]:
class AlkylMonomer(mb.Compound):
    """Creates a monomer for an alkyl chain"""
    def __init__(self):
        super(AlkylMonomer, self).__init__()
        self.add(mb.Particle(name='C', pos=[0,0,0]), label='C[$]')
        self.add(mb.Particle(name='Br', pos=[-0.109,0,0]), label='BrC[$]')
        self.add(mb.Particle(name='H', pos=[0.109,0,0]), label = 'HC[$]')
        theta = 0.5 * (180 - 109.5) * np.pi / 180
        self['BrC'][0].rotate(theta, around=[0,1,0])
        self['HC'][0].rotate(-theta, around=[0,1,0])
        self.add_bond((self[0],self['BrC'][0]))
        self.add_bond((self[0],self['HC'][0]))

        self.add(mb.Port(anchor=self[0]), label='up')
        self['up'].translate([0, -0.154/2, 0])
        self['up'].rotate(theta, around=[1,0,0])
        self.add(mb.Port(anchor=self[0]), label='down')
        self['down'].translate([0, 0.154/2, 0])
        self['down'].rotate(np.pi, around=[0,1,0])
        self['down'].rotate(-theta, around=[1,0,0])

    

In [3]:
import itertools as it
import numpy as np
import mbuild as mb

from mbuild.coordinate_transform import force_overlap
from mbuild.utils.validation import assert_port_exists
from mbuild import clone


__all__ = ['Polymer']


class Polymer(mb.Compound):
    """Connect one or more components in a specified sequence.
    Parameters
    ----------
    monomers : mb.Compound or list of mb.Compound
        The compound(s) to replicate.
    n : int
        The number of times to replicate the sequence.
    sequence : str, optional, default='A'
        A string of characters where each unique character represents one
        repetition of a monomer. Characters in `sequence` are assigned to
        monomers in the order assigned by the built-in `sorted()`.
    port_labels : 2-tuple of strs, optional, default=('up', 'down')
        The names of the two ports to use to connect copies of proto.
    """
    def __init__(self, monomers, n, sequence='A', port_labels=('up', 'down')):
        if n < 1:
            raise ValueError('n must be 1 or more')
        super(Polymer, self).__init__()
        if isinstance(monomers, mb.Compound):
            monomers = (monomers,)
        for monomer in monomers:
            for label in port_labels:
                assert_port_exists(label, monomer)

        unique_seq_ids = sorted(set(sequence))

        if len(monomers) != len(unique_seq_ids):
            raise ValueError('Number of monomers passed to `Polymer` class must'
                             ' match number of unique entries in the specified'
                             ' sequence.')

        # 'A': monomer_1, 'B': monomer_2....
        seq_map = dict(zip(unique_seq_ids, monomers))

        last_part = None
        for n_added, seq_item in enumerate(it.cycle(sequence)):
            this_part = clone(seq_map[seq_item])
            self.add(this_part, 'monomer[$]')
            if last_part is None:
                first_part = this_part
            else:
                # Transform this part, such that it's bottom port is rotated
                # and translated to the last part's top port.
                force_overlap(this_part,
                              this_part.labels[port_labels[1]],
                              last_part.labels[port_labels[0]])
            last_part = this_part
            if n_added == n * len(sequence) - 1:
                break

        # Hoist the last part's top port to be the top port of the polymer.
        self.add(last_part.labels[port_labels[0]], port_labels[0], containment=False)

        # Hoist the first part's bottom port to be the bottom port of the polymer.
        self.add(first_part.labels[port_labels[1]], port_labels[1], containment=False)

        

In [4]:
class OH(mb.Compound):
    """Creates a hydroxyl group"""
    def __init__(self):
        super(OH,self).__init__()
        self.add(mb.Particle(name='OA', pos = [0,0,0]), label='O')
        self.add(mb.Particle(name='H', pos = [.096, 0, 0]), label='H')
        self.add_bond((self[0], self[1]))

        self.add(mb.Port(anchor=self[0], orientation=[-1,
            np.tan(72*np.pi/180), 0], separation=.135/2), label='up')

class COOH(mb.Compound):
    """Creates headgroup of a carboxylic acid"""
    def __init__(self, ester = None):
        super(COOH,self).__init__()
        self.add(mb.Particle(name='C'), label='C')
        self.add(mb.Particle(name='O', pos = [0, .123, 0]),label='O[$]')
        self.add_bond((self['C'],self['O'][0]))
        
        self.add(mb.Port(anchor=self[0],
            orientation=[-1,-np.tan(34.5*np.pi/180),0],
            separation=.132/2), label='up')
        
        if ester:
            self.add(mb.Particle(name='O'), label='O[$]')
            self['O'][1].translate([.15,0,0])
            theta = ((180-111) / 2) * np.pi / 180
            self['O'][1].rotate(-theta, around=[0,0,1])
            self.add_bond((self['C'],self['O'][1]))

            self.add(mb.Port(anchor=self['O'][1],
                orientation=[1,np.tan(52.5*np.pi/180),0],
                separation=.14/2), label='down')
        else:
            self.add(mb.Port(anchor=self[0], 
                orientation=[1,-np.tan(32*np.pi/180), 0], 
                separation=.132/2),label='down')
            self.add(OH(),label='OH')
            mb.force_overlap(move_this=self['OH'],
                    from_positions=self['OH']['up'],
                    to_positions=self['down'])
            
class Methane(mb.Compound):
    def __init__(self):
        super(Methane, self).__init__()
        carbon = mb.Particle(name='C')
        self.add(carbon)

        hydrogen = mb.Particle(name='H', pos=[0.1, 0, -0.07])
        self.add(hydrogen)

        self.add_bond((self[0], self[1]))

        self.add(mb.Particle(name='O', pos=[-0.1, 0, -0.07]))
        self.add(mb.Particle(name='F', pos=[0, 0.1, 0.07]))
        self.add(mb.Particle(name='N', pos=[0, -0.1, 0.07]))

        self.add_bond((self[0], self[2]))
        self.add_bond((self[0], self[3]))
        self.add_bond((self[0], self[4]))

from mbuild.lib.moieties.ch3 import CH3

class FFA(mb.Compound):
    """Creates a saturated free fatty acid of n carbons based on user
    input"""
    def __init__(self, chain_length, ester=True):
        super(FFA, self).__init__()
        
        if ester:
            self.add(COOH(ester=True), label='head')
        else:
            self.add(COOH(), label='head')
        
        tail = mb.Polymer(AlkylMonomer(), (chain_length - 2))
        self.add(tail, label='tail')
        mb.x_axis_transform(self['tail'], new_origin=self['tail'][0],
                point_on_x_axis=self['tail'][6],
                point_on_xy_plane=self['tail'][3])
        self.add(CH3(), label='tailcap')
        
        self['head'].rotate(np.pi, [0,1,0])
        self['head'].translate([-self['tail'][3].pos[0],
            self['tail'][3].pos[1], 0])
        mb.force_overlap(move_this=self['tailcap'],
                from_positions=self['tailcap']['up'],
                to_positions=self['tail']['up'])
        
        self['head']['up'].spin(-np.pi/2, self['head'][0].pos)
        mb.force_overlap(move_this=self['head'],
                from_positions=self['head']['up'],
                to_positions=self['tail']['down'])
        self.spin(np.pi/2, [0,1,0])

In [5]:
import numpy as np
from copy import deepcopy
pp = Polymer(AlkylMonomer(), n= 4)
oo = deepcopy(pp)
# for ii, jj in zip(pp,oo):
#     print(ii)
#     print(jj)
#pp.save("pollywannacracker.mol2", overwrite= True)
from coordinate_transform import _mirror
# pp.mirror(about= 'yz')
#pp.save("pollywannaflipper.mol2", overwrite= True)
# for ii,jj in zip(oo.children,pp.children):
#     if np.allclose(ii,jj, atol=1e-13):
#         print('yessir')
#     else:
#         print('drat')
# for ii, jj in zip(pp,oo):
#     print(ii)
#     print(jj)
# going to need to make a chiral molecule

In [6]:
mm = Methane()
# for ii in mm.children:
#     print(ii)
#mm.save('monowannacrack.mol2', overwrite=True)
# mm.mirror()
# for ii in mm.children:
#     print(ii)
#mm.save('monowannaflip.mol2', overwrite=True)
# mm.mirror()
#mm.save('monoflipback.mol2', overwrite=True)


In [7]:
dim = 3
edge_lengths = [.3359, .3359, .3359]
lattice_vecs = [[1,0,0], [0,1,0], [0,0,1]]
basis = {'origin':[[0,0,0]]}
simple_cubic = mb.Lattice(edge_lengths, 
                          lattice_vectors=lattice_vecs, dimension=dim, 
                          basis_atoms=basis)
compound_dictionary = {'origin': mm}
crystal_modimethane = simple_cubic.populate(compound_dict=compound_dictionary, x=3, y=3, z=3)
#crystal_modimethane.save('crystal_modimethane.mol2', overwrite= True)
#crystal_modimethane.mirror(child_chirality=True, looking_for=[''])
#crystal_modimethane.save('crystal_modimethane_child_chirality.mol2', overwrite= True)



/home/witteaj/builds/mbuild/mbuild/utils/decorators.py:11: DeprecationWarning: translate is deprecated. Please use Compound.translate()
  warn(printed_message, DeprecationWarning)
/home/witteaj/builds/mbuild/mbuild/utils/decorators.py:11: DeprecationWarning: translate is deprecated. Please use Compound.translate()
  warn(printed_message, DeprecationWarning)
/home/witteaj/builds/mbuild/mbuild/utils/decorators.py:11: DeprecationWarning: translate is deprecated. Please use Compound.translate()
  warn(printed_message, DeprecationWarning)
/home/witteaj/builds/mbuild/mbuild/utils/decorators.py:11: DeprecationWarning: translate is deprecated. Please use Compound.translate()
  warn(printed_message, DeprecationWarning)
/home/witteaj/builds/mbuild/mbuild/utils/decorators.py:11: DeprecationWarning: translate is deprecated. Please use Compound.translate()
  warn(printed_message, DeprecationWarning)
/home/witteaj/builds/mbuild/mbuild/utils/decorators.py:11: DeprecationWarning: translate is deprecat

In [8]:
from copy import deepcopy
from warnings import warn

import numpy as np

import mbuild as mb

__all__ = ['Monolayer']


class Monolayer(mb.Compound):
    """A general monolayer recipe.
    Parameters
    ----------
    surface : mb.Compound
        Surface on which the monolayer will be built.
    chains : list of mb.Compounds
        The chains to be replicated and attached to the surface.
    fractions : list of floats
        The fractions of the pattern to be allocated to each chain.
    backfill : list of mb.Compound, optional, default=None
        If there are fewer chains than there are ports on the surface,
        copies of `backfill` will be used to fill the remaining ports.
    pattern : mb.Pattern, optional, default=mb.Random2DPattern
        An array of planar binding locations. If not provided, the entire
        surface will be filled with `chain`.
    tile_x : int, optional, default=1
        Number of times to replicate substrate in x-direction.
    tile_y : int, optional, default=1
        Number of times to replicate substrate in y-direction.
    """

    def __init__(self, surface, chains, fractions=None, backfill=None, pattern=None,
                 tile_x=1, tile_y=1, **kwargs):
        super(Monolayer, self).__init__()

        # Replicate the surface.
        tiled_compound = mb.TiledCompound(surface, n_tiles=(tile_x, tile_y, 1))
        self.add(tiled_compound, label='tiled_surface')

        if pattern is None:  # Fill the surface.
            pattern = mb.Random2DPattern(len(tiled_compound.referenced_ports()))

        if isinstance(chains, mb.Compound):
            chains = [chains]
        
        if fractions:
            fractions = list(fractions)
            if len(chains) != len(fractions):
                raise ValueError("Number of fractions does not match the number"
                                 " of chain types provided")

            n_chains = len(pattern.points)

            # Attach chains of each type to binding sites based on
            # respective fractions.
            for chain, fraction in zip(chains[:-1], fractions[:-1]):

                # Create sub-pattern for this chain type
                subpattern = deepcopy(pattern)
                n_points = int(round(fraction * n_chains))
                warn("\n Adding {} of chain {}".format(n_points, chain))
                pick = np.random.choice(subpattern.points.shape[0], n_points,
                                        replace=False)
                points = subpattern.points[pick]
                subpattern.points = points

                # Remove now-occupied points from overall pattern
                pattern.points = np.array([point for point in pattern.points.tolist()
                                           if point not in subpattern.points.tolist()])

                # Attach chains to the surface
                attached_chains, _ = subpattern.apply_to_compound(
                    guest=chain, host=self['tiled_surface'], backfill=None, **kwargs)
                self.add(attached_chains)

        else:
            warn("\n No fractions provided. Assuming a single chain type.")

        # Attach final chain type. Remaining sites get a backfill.
        warn("\n Adding {} of chain {}".format(len(pattern), chains[-1]))
        attached_chains, backfills = pattern.apply_to_compound(guest=chains[-1],
                         host=self['tiled_surface'], backfill=backfill, **kwargs)
        self.add(attached_chains)
        self.add(backfills)


In [9]:
ff = FFA(chain_length=6)

In [10]:
dim = 3
edge_lengths = [.3359, .3359, .3359]
lattice_vecs = [[1,0,0], [0,1,0], [0,0,1]]
basis = {'origin':[[0,0,0]]}
ffa_lattice = mb.Lattice(edge_lengths, 
                          lattice_vectors=lattice_vecs, dimension=dim, 
                          basis_atoms=basis)
ffa_crystal = ffa_lattice.populate(compound_dict={'origin': ff}, x=3, y=2, z=2)
# ffa_crystal.mirror(child_chirality= True, looking_for= "COOH", override= True)
print(ffa_crystal[0].particles)
print(ffa_crystal[0].xyz_with_ports)


<bound method Compound.particles of <C pos=( 0.2006, 0.0818, 0.3212), 0 bonds, id: 140185602259768>>
[ 0.20064327  0.08181973  0.32123805]


/home/witteaj/builds/mbuild/mbuild/utils/decorators.py:11: DeprecationWarning: translate is deprecated. Please use Compound.translate()
  warn(printed_message, DeprecationWarning)
/home/witteaj/builds/mbuild/mbuild/utils/decorators.py:11: DeprecationWarning: translate is deprecated. Please use Compound.translate()
  warn(printed_message, DeprecationWarning)
/home/witteaj/builds/mbuild/mbuild/utils/decorators.py:11: DeprecationWarning: translate is deprecated. Please use Compound.translate()
  warn(printed_message, DeprecationWarning)
/home/witteaj/builds/mbuild/mbuild/utils/decorators.py:11: DeprecationWarning: translate is deprecated. Please use Compound.translate()
  warn(printed_message, DeprecationWarning)
/home/witteaj/builds/mbuild/mbuild/utils/decorators.py:11: DeprecationWarning: translate is deprecated. Please use Compound.translate()
  warn(printed_message, DeprecationWarning)
/home/witteaj/builds/mbuild/mbuild/utils/decorators.py:11: DeprecationWarning: translate is deprecat

In [11]:
# dim = 3
# edge_lengths = [.5359, .5359, .5359]
# lattice_vecs = [[1,0,0], [0,1,0], [0,0,1]]
# basis = {'origin':[[0,0,0]]}
# simple_cubssuck = mb.Lattice(edge_lengths, 
#                           lattice_vectors=lattice_vecs, dimension=dim, 
#                           basis_atoms=basis)
# compound_dictionary = {'origin': pp}
# biggs = simple_cubssuck.populate(compound_dict=compound_dictionary, x=3, y=3, z=3)
# biggs.save('lattice_of_polymers.mol2', overwrite= True)
ff = FFA(chain_length=6)
spacings = [.94123, .94123, .94123]
basis = {'mm' : [[0., 0., 0.]], 'ff' : [[.5, .5, .5]]}
mixed_latty = mb.Lattice(spacings, basis_atoms=basis, dimension=3)
mixdix = {"mm" : mm, "ff" : ff}
mixed_crystal = mixed_latty.populate(x=2,y=2,z=3, compound_dict= mixdix)
OG = mb.clone(mixed_crystal)
OG2 = mb.clone(mixed_crystal)
OG3 = mb.clone(mixed_crystal)
OG4 = mb.clone(mixed_crystal)
#mixed_crystal.save('mixed_lattice_ff_mm.mol2', overwrite = True)
mixed_crystal.mirror(override= True)
#mixed_crystal.save('mixed_lattice_ff_mm_reg_mirror.mol2', overwrite = True)
mixed_crystal.mirror(override= True)
#mixed_crystal.save("mixed_lattice_ff_mm_flipped_back.mol2", overwrite= True)

mixed_crystal.mirror(child_chirality= True, looking_for="Methane", override= True)
#mixed_crystal.save("mixed_lattice_mm_ff_w_mm_chirality.mol2", overwrite=True)
OG.mirror(about= 'xz', child_chirality= True, looking_for= "AlkylMonomer", override= True)
#OG.save("mixed_lattice_mm_ff_w_Alkyl_chirality1.mol2", overwrite=True)
OG3.mirror(about= 'xz', child_chirality= True, looking_for= "Polymer", override= True)
#OG3.save("mixed_lattice_mm_ff_w_polymer_chirality.mol2", overwrite=True)
lookin = 'Polymer'
missing = deepcopy(lookin)
from coordinate_transform import _translate
# for subc in OG2.subcompounds_by_name(lookin):
#     print('Y')
#     if not subc:
#         continue
#     if subc.name in missing:
#         missing.remove(subc.name)
#     print(subc.name)
#     for ii in subc:
#         print('mooove')
#         print(ii)
#         for jj in subc.root.bond_graph.neighbors(ii):
#             print(jj.parent.name)
#             print(jj)
#             print('')
#         print('')
#     #print(OG2.root.bond_graph.neighbors(subc))
# #     for kk in subc.referenced_ports():
# #         print('i')
# #         if kk.parent.parent.name != 'CH#':
# #             print(kk.parent.name)
# #             print('')
# #         else:
# #             print('the one that got away', kk.parent.name)
# #     #subc.translate(np.array([.2,.2,.2]))
#     if len(missing) > 0:
#         raise ValueError("One or more of the following names do not "
#                          "exist in {}. Missing: {}.".format(OG2.name, missing))
# #OG2.save("mixed_lattice_mm_ff_move_to_see_if_we_touch_subcompounds.mol2", overwrite=True)
# # previous line indicates that we are indeed modifying the subcompounds

OG4.mirror(override= True, looking_for= 'AlkylMonomer', child_chirality= True,
           keep_position_with_neighbor= ['CH3'], within= 'Polymer',
           sees_twin_label = [0,1,2,3,4,5])

/home/witteaj/builds/mbuild/mbuild/utils/decorators.py:11: DeprecationWarning: translate is deprecated. Please use Compound.translate()
  warn(printed_message, DeprecationWarning)
/home/witteaj/builds/mbuild/mbuild/utils/decorators.py:11: DeprecationWarning: translate is deprecated. Please use Compound.translate()
  warn(printed_message, DeprecationWarning)
/home/witteaj/builds/mbuild/mbuild/utils/decorators.py:11: DeprecationWarning: translate is deprecated. Please use Compound.translate()
  warn(printed_message, DeprecationWarning)
/home/witteaj/builds/mbuild/mbuild/utils/decorators.py:11: DeprecationWarning: translate is deprecated. Please use Compound.translate()
  warn(printed_message, DeprecationWarning)
/home/witteaj/builds/mbuild/mbuild/utils/decorators.py:11: DeprecationWarning: translate is deprecated. Please use Compound.translate()
  warn(printed_message, DeprecationWarning)
/home/witteaj/builds/mbuild/mbuild/utils/decorators.py:11: DeprecationWarning: translate is deprecat

NameError: name 'recenter' is not defined

In [22]:
for o in list(OG, OG2):
    o.translate([5,6,7])



print(list(OG.find_particle_in_path(within_path=['Br', "AlkylMonomer","FFA[0]"])))
print("")
print(list(OG2.find_particle_in_path(within_path=['Br', "AlkylMonomer","FFA[0]"])))



[<Br pos=( 0.5814, 0.4077, 0.6746), 0 bonds, id: 140185596060784>, <Br pos=( 0.7594, 0.6224, 0.5488), 0 bonds, id: 140185596063696>, <Br pos=( 0.5814, 0.4077, 0.4231), 0 bonds, id: 140185596111728>, <Br pos=( 0.7594, 0.6224, 0.2973), 0 bonds, id: 140185596167952>]

[<Br pos=( 0.5814, 0.4077, 0.6746), 0 bonds, id: 140185593256536>, <Br pos=( 0.7594, 0.6224, 0.5488), 0 bonds, id: 140185593316856>, <Br pos=( 0.5814, 0.4077, 0.4231), 0 bonds, id: 140185592848792>, <Br pos=( 0.7594, 0.6224, 0.2973), 0 bonds, id: 140185592851704>]


In [ ]:
n=0

for ii in OG.children:
    #print(ii.referenced_ports)
    for jj in ii.children:
        pass
#         for kk in jj.referenced_ports():
#             if kk.parent.parent.name != 'FFA':
#                 print(kk.parent.name)
#                 print('')
#             else:
#                 print('the one that got away', kk)
# OG.bonds()
# dum = []
for subc in OG.subcompounds_by_name(looking_for=['Polymer']):
    if subc:
        print(subc)
        for jj in subc.parent.bonds():
            print(jj)
            print("2tups")
            for tup in jj:
                print("go head")
                for anny in tup.ancestors():
                    print("meet the aunties")
                    print(anny)
                    if anny is subc:
                        print("we ok")
                        break
                else:
                    print("naw")
                    print(tup)
                    print(jj)

                    print(ii)

        print("")
#         for jj in subc:
#             for kk in subc.root.bond_graph.neighbors(jj):
#                 #print(jj.parent['O[0]'] in ii)
#                 #if kk.parent['C']:
#                 for anny in kk.ancestors():
#                     if anny is subc:
#                         print("is subc")
#                         print("anny")
#                         print(anny)
#                         print("subc")
#                         print(subc)
#                         print(" ")
#                         break
#                     elif anny.name == subc.name:
#                         print("no good")
#                         print("anny")
#                         print(anny)
#                         print("subc")
#                         print(subc)
#                         print(" ")


#                         print(anny)
#                         if anny.name == subc.name and anny is not subc:
#                             print('ay')
#                             print(subc)
#                             print('subc^')
#                             break
#                         else:
#                             print("shit")
#                 break
                #print(kk == jj.parent['O[0]'])
# OG.labels
# for rr in OG.subcompounds_by_name(looking_for=["CH3"]):
#     dum 
#     if rr:
        


In [ ]:
for level1 in OG4.children:
    print("level1")
    print(level1)
    print(level1.name)
    print(" ")
    for level2 in level1.children:
        print("level2")
        print(level2)
        print(level2.name)
        print(" ")
        for level3 in level2.children:
            print("level3")
            print(level3)
            print(level3.name)
            print(" ")
    

In [ ]:
# for ii in OG4.subcompounds_by_name(looking_for=["AlkylMonomer"]):
#     nn = list(OG4.subcompounds_by_name(looking_for=["AlkylMonomer"]))
#     yield from[jj for jj in nn if jj is ii]
for ii in OG4:
    for jj in ii:
        print(jj is ii)
# for tt in wild():
#     if tt:
#         print(tt)

#     print(len(list(OG4.particles_by_name("C"))))
#     print('chiiiiiil\n\n\n\n')

In [ ]:
# from random import shuffle, random, uniform
# import numpy as np
# import argparse

# import mbuild as mb
# from mbuild import clone


# class H2O(mb.Compound):
#     """A water molecule. """
#     def __init__(self):
#         super(H2O, self).__init__()

#         self.add(mb.Particle(name='OW', pos=[1.0203, 0.7604, 1.2673]))
#         self.add(mb.Particle(name='HW1', pos=[0.9626, 0.8420, 1.2673]))
#         self.add(mb.Particle(name='HW2', pos=[0.9626, 0.6787, 1.2673]))
#         self.add_bond((self[0], self[1]))
#         self.add_bond((self[0], self[2]))
#         self.name = 'HOH'


# class Bilayer(mb.Compound):
#     """The Bilayer Builder creates a lipid bilayer, solvates it, and stores it as an mBuild Compound. 
#     Because the Bilayer Builder does not check for the orientation of the mBuild molecules the user 
#     provides from a previously created class or file, the user must ensure the following things are true 
#     about the lipids they input to the builder:
#         - The lipid's longitudinal axis lies on the z-axis of the Cartesian coordinate system
#         - The lipid's reference atom is located at the origin (see ref_atoms below)
#         - The rest of the lipid is pointing in the negative z direction
        
#     The user may input the fraction of each lipid in the bilayer, as well as a number of important bilayer 
#     properties such as area per lipid and tilt angle.
#     Parameters
#     ----------
#     lipids : list of tuple
#         List of tuples in format (lipid, frac, z-offset) where frac is
#         the fraction of that lipid in the bilayer (lipid is a
#         Compound), and z-offset is the distance in nanometers from the
#         headgroup to the lipid-water interface
#     ref_atoms : list
#         Indices of the atom in lipids to form the solvent interface.
#         By definition, this atom denotes the headgroup of the lipid.
#     args : NameSpace
#         The parser passed by the argparse module; allows for command-line integration with
#         the Bilayer builder.
#     max_tail_randomization : float, optional, default=0.0
#         For a set tilt angle, the maximum amount each lipid may be
#         randomly spun around the z-axis.
#     solvent : Compound, optional, default=H2O()
#         Compound to solvate the bilayer with. Typically, a
#         pre-equilibrated box of solvent.
#     lipid_box : Box, optional
#         A Box containing the lipids where no solvent will be added.
#     unit_conversion : int, optional, default=1
#         Adjust for unit conversions if necessary
        
#     Attributes
#     ----------
#     n_lipids_x : int
#         Number of lipids in the x-direction per layer.
#     n_lipids_y : int
#         Number of lipids in the y-direction per layer.
#     lipids : list
#         List of tuples in format (lipid, frac, z-offset) where frac is
#         the fraction of that lipid in the bilayer (lipid is a
#         Compound), and z-offset is the distance in nanometers from the
#         headgroup to the lipid-water interface
#     ref_atoms : int
#         Indices of the atom in lipids to form the solvent interface
#     lipid_box : Box, optional
#         A Box containing the lipids where no solvent will be added.
#     solvent : Compound
#         Compound to solvate the bilayer with. Typically, a
#         pre-equilibrated box
#         of solvent.
#     solvent_per_lipid : int, optional, default=0
#         Number of solvent molecules per lipid
#     unit_conversion : int, optional, default=1
#         Adjust for unit conversions if necessary
#     """
    
#     def __init__(self, lipids, args, ref_atoms, itp_path, max_tail_randomization=0, solvent=H2O(),
#                  lipid_box=None, solvent_per_lipid=0, cross_tilt=False, unit_conversion=1,
#                  filename=None):
#         super(Bilayer, self).__init__()

#         self._sanitize_inputs(lipids, ref_atoms)

#         self.n_lipids_x = args.n_lipids_x
#         self.n_lipids_y = args.n_lipids_y
#         self.lipids = lipids
#         if args.apl:
#             area_per_lipid = args.apl
#         else:
#             area_per_lipid = 0.3    # Set default apl equal to 0.3 nm^2

#         self.ref_atoms = ref_atoms
#         self._lipid_box = lipid_box
#         self.unit_conversion = unit_conversion
#         if filename:
#             self.filename = filename
#         else:
#             self.filename = ''
#             components = [str(lipid[0].name) + '_' + str(lipid[1]) + '_' for lipid in lipids]
#             for component in components:
#                 self.filename += component
#             self.filename += 'UA'
#             # self.filename += 'AA' TODO: Make a command line flag
#             # self.filename += 'CG' TODO: Same as above

#         # Set the 2D lipid locations on a grid pattern
#         self._set_grid_pattern(area_per_lipid)
#         if args.spacing:
#             z_spacing = args.spacing
#         else:
#             z_spacing = 0.4
        
#         # Set other important geometric attributes of the lipids
#         if args.tilt_angle:
#             self.tilt = args.tilt_angle * np.pi / 180
#         else:
#             self.tilt = 0.0
#         self.z_orientation = max_tail_randomization

#         # Solvent attributes.
#         self.solvent = solvent
#         self.solvent_per_lipid = args.solvent
        
#         # Private attributes for getter methods 
#         self._number_of_each_lipid_per_layer = []
#         self._solvent_per_layer = None
        
#         # Path to .itp files
#         self.itp_path = itp_path

#         # Write topology file header
#         print('Writing <{0}> ...'.format(self.filename + '.top'))
#         top_file = self.write_top_header(self.filename)

#         # Create lipid leaflets and add them to the bilayer compound 
#         # structure.
#         lipid_bilayer = mb.Compound()
#         solvent_components = mb.Compound()
#         top_file, top_leaflet, top_lipid_labels = self.create_layer(top_file)
#         lipid_bilayer.add(top_leaflet, label='top_leaflet')
#         if args.mirror:
#             top_file, bottom_leaflet, bottom_lipid_labels = self.create_layer(top_file, lipid_indices=top_lipid_labels)
#         else:
#             top_file, bottom_leaflet, bottom_lipid_labels = self.create_layer(top_file)
#         lipid_bilayer.add(bottom_leaflet, label='bottom_leaflet')
        
#         # Translate and rotate the bottom leaflet
#         self._translate_bottom_leaflet(lipid_bilayer, z_spacing,
#                                        args.cross_tilt)
        
#         # solvate the lipids
#         if solvent_per_lipid > 0:
#             top_file, solvent_components = self.solvate_bilayer(top_file,
#                                                                 lipid_bilayer, solvent_components)
#         # close the completed topology file
#         top_file.close()

#         # add everything to the overall Compound structure
#         self.add(lipid_bilayer, label='lipid_bilayer')
#         if solvent_per_lipid > 0:
#             self.add(solvent_components, label='solvent')
        
#     def create_layer(self, top_file, lipid_indices=None):
#         """Create a monolayer of lipids.
#         Parameters
#         ----------
#         top_file : file
#             A topology file into which molecule information will be
#             written for simulation
#         lipid_indices : list, optional, default=None
#             A list of indices associated with each lipid in the layer.
#         Returns
#         -------
#         layer : mb.Compound
#             An mBuild compound containing the entire leaflet of
#             molecules
#         lipid indices : list
#             The indices for each lipid corresponding to its location
#             on the grid; necessary if user desires a mirror effect between
#             the top and bottom leaflets
        
#         """
#         layer = mb.Compound()
#         if not lipid_indices:
#             lipid_indices = list(range(self.lipids_per_leaflet))
#             shuffle(lipid_indices)
            
#         n_previous_types = 0
#         for n_type, n_of_lipid_type in enumerate(
#                 self.number_of_each_lipid_per_leaflet):
#             current_type = self.lipids[n_type][0]
            
#             # Write to topology file
#             if self.number_of_each_lipid_per_leaflet[n_type] != 0:
#                 top_file.write("{:<10s}{:<10d}\n".format(
#                     self.lipids[n_type][0].name,
#                     self.number_of_each_lipid_per_leaflet[n_type]))

#             # Loop through the leaflet's entire quantity of one lipid
#             # type
#             for n_this_type in range(n_of_lipid_type):
#                 lipids_placed = n_previous_types + n_this_type
#                 new_lipid = clone(current_type)
#                 random_index = lipid_indices[lipids_placed]
#                 position = self.pattern[random_index]

#                 # Zero and space in z-direction, apply necessary geometric
#                 # transformations
#                 particles = list(new_lipid.particles())
#                 ref_atom = self.ref_atoms[n_type]
#                 new_lipid.spin(-self.tilt, [0, 1, 0])
#                 new_lipid.spin(theta=uniform(-self.z_orientation,
#                                              self.z_orientation) * np.pi/180, around=[0, 0, 1])
#                 new_lipid.translate(-particles[ref_atom].pos + [0, 0, self.lipids[n_type][2]])
                
#                 # Move to point on pattern and add the lipid to the
#                 # leaflet compound
#                 new_lipid.translate(position)
#                 layer.add(new_lipid)
                
#             n_previous_types += n_of_lipid_type
#         return top_file, layer, lipid_indices
    
#     def _translate_bottom_leaflet(self, lipid_bilayer, spacing_z, args):
#         mins = np.amin(lipid_bilayer['top_leaflet'].xyz, axis=0)
#         z_min = mins[2]
#         spacing_z *= self.unit_conversion
#         lipid_bilayer['bottom_leaflet'].translate([0, 0, (z_min - spacing_z)])
#         lipid_bilayer['bottom_leaflet'].spin(np.pi, [1, 0, 0])
#         if not args.cross_tilt:
#             lipid_bilayer['bottom_leaflet'].spin(np.pi, [0, 0, 1])

#     def solvate_bilayer(self, top_file, lipid_bilayer, solvent_components):
#         """Solvate the constructed bilayer. 
        
#         Parameters
#         ----------
#         top_file : file
#             Topology file into which solvent information is written
#         lipid_bilayer : mb.Compound
#             An mBuild compound containing both leaflets of the bilayer
#         solvent_components : mb.Compound
#             A (temporarily) empty Compound that will serve as the container
#             for the solvated boxes
        
#         Returns
#         -------
#         top_file : file
#             Updated topology file
#         solvent_components : mb.Compound
#             The container for the solvated boxes created in this function
#         """
#         water_volume = .0299 * np.power(self.unit_conversion, 3) 
#         # TODO: Support other solvents' volumes in a user-friendly way
        
#         # Calculate box dimension
#         box_volume = water_volume * self.solvent_per_layer
#         x_min_box = min(lipid_bilayer.xyz[:, 0])
#         x_max_box = max(lipid_bilayer.xyz[:, 0])
#         y_min_box = min(lipid_bilayer.xyz[:, 1])
#         y_max_box = max(lipid_bilayer.xyz[:, 1])
        
#         box_area = (x_max_box - x_min_box) * (y_max_box - y_min_box)
#         box_height = box_volume / box_area
        
#         z_min_top = max(lipid_bilayer.xyz[:, 2])
#         z_max_top = z_min_top + box_height
        
#         z_max_bottom = min(lipid_bilayer.xyz[:, 2])
#         z_min_bottom = z_max_bottom - box_height

#         top_solvent_box = mb.Box(mins=[x_min_box, y_min_box, z_min_top],
#                                  maxs=[x_max_box, y_max_box, z_max_top])
        
#         bottom_solvent_box = mb.Box(mins=[x_min_box, y_min_box, z_min_bottom],
#                                     maxs=[x_max_box, y_max_box, z_max_bottom])

#         solvent_components.add(mb.fill_region(compound=[self.solvent, self.solvent],
#                                               n_compounds=[self.solvent_per_layer, self.solvent_per_layer],
#                                               region=[top_solvent_box, bottom_solvent_box]))

#         # Write solvent components to topology file
#         top_file.write("{:<10s}{:<10d}\n".format('SOL',
#                                                  self.solvent_per_layer * 2))

#         return top_file, solvent_components
            
#     def write_top_header(self, filename='bilayer_test'):
#         """ Generate topology file header based on existing .itp files
#         Parameters
#         ----------
#         filename : str, optional, default = default
#             Filename for the topology file
#         Returns
#         -------
#         top_file
#             The open topology file to be written into by the rest of
#             the Bilayer Builder
#         NOTE: Function assumes a particular path for itp files; use
#         constant ITP_PATH to customize
#         """
#         # TODO: create flag for user to input a custom .itp path

#         top_file = open(filename + '.top', 'w')

#         # Include statements to locate .itp files for .top creation
#         top_file.write(";#include \"{}ff.itp\"\n".format(self.itp_path))
#         top_file.write("#include \"{}ff_b.itp\"\n".format(self.itp_path))
#         top_file.write("; SPC water topology \n")
#         top_file.write(";#include \"{}spc.itp\"\n".format(self.itp_path))
#         top_file.write("#include \"{}spc_b.itp\"\n".format(self.itp_path))
#         top_file.write("\n[ system]\n")
#         top_file.write("United-atom bilayer system\n")
#         # top_file.write("All-atom bilayer system\n")
#         # top_file.write("Coarse-grained bilayer system\n")
#         # TODO: Create flags for above system options
#         top_file.write("\n[ molecules ]\n")

#         return top_file

#     @property
#     def solvent_per_layer(self):
#         """Determine the number of solvent molecules per single layer.  """
#         return self.n_lipids_x * self.n_lipids_y * self.solvent_per_lipid

#     @property
#     def number_of_each_lipid_per_leaflet(self):
#         """The number of each lipid per leaflet. """
#         if self._number_of_each_lipid_per_layer:
#             return self._number_of_each_lipid_per_layer

#         for lipid in self.lipids[:-1]:
#             self._number_of_each_lipid_per_layer.append(int(round(lipid[1] * self.lipids_per_leaflet)))

#         # TODO: give warning if frac * n different than actual
#         # Rounding errors may make this off by 1, so just do total - whats_been_added.
#         self._number_of_each_lipid_per_layer.append(self.lipids_per_leaflet
#                                                     - sum(self._number_of_each_lipid_per_layer))
#         assert len(self._number_of_each_lipid_per_layer) == len(self.lipids)
#         return self._number_of_each_lipid_per_layer

#     @staticmethod
#     def _sanitize_inputs(lipids, ref_atoms):
#         """Check for proper inputs
    
#         Ensure that the user's lipid fractions add up to 1, 
#         or raise a ValueError.
    
#         Ensure that the user has input the same number of reference
#         atoms and lipid components, or raise an AssertionError."""
    
#         if sum([lipid[1] for lipid in lipids]) != 1.0:
#             raise ValueError('Lipid fractions do not add up to 1.')
#         assert len(ref_atoms) == len(lipids), "Please provide one reference atom for each lipid"
        
#     def _set_grid_pattern(self, area_per_lipid):
#         """Utilize an mBuild 2DGridPattern to create the scaffold of points that the lipids will be laid onto"""
        
#         self.lipids_per_leaflet = self.n_lipids_x * self.n_lipids_y
#         pattern = mb.Grid2DPattern(self.n_lipids_x, self.n_lipids_y)
#         pattern.scale(np.sqrt(area_per_lipid * self.lipids_per_leaflet) * self.unit_conversion)


In [ ]:
# bb = Bilayer(lipids= [(pp, .2, 0), (mm, .8, 0)], ref_atoms= [0,0,0])